# Ipytone: a simple oscillator example

A basic example of using ipywidgets with [Tone.js](https://tonejs.github.io/).

See https://tonejs.github.io/examples/oscillator.html.

In [ ]:
import ipytone
import ipywidgets
from IPython.display import display

Create the oscillator widget (note: this widget has no visible output).

In [ ]:
osc = ipytone.Oscillator()

If we want to hear the sound generated by the oscillator, we need to connect it to the "destination" node through which the audio signal is sent to the speakers or headphones.

In [ ]:
osc.to_destination()

Let's check the audio graph (no fancy view available yet).

In [ ]:
graph = ipytone.get_audio_graph()

graph.nodes

In [ ]:
graph.connections

Add some other widgets to control the oscillator.

Note: the `osc` widget created above must be linked to another, visible widget in order to activate it, since most Web Audio API implementations require sounds to be triggered from an explicit user action. 

In [ ]:
freq_slider = ipywidgets.FloatSlider(
    value=440,
    min=100,
    max=1000,
    step=1,
    description='Frequency',
    readout=True,
    readout_format='.1f',
)

detune_slider = ipywidgets.FloatSlider(
    value=0,
    min=-100,
    max=100,
    step=1,
    description='Detune',
    readout=True,
)

vol_slider = ipywidgets.FloatSlider(
    value=-16,
    min=-50,
    max=-10,
    step=1,
    description='Volume',
    readout=True,
    readout_format='.1f',
)

osc_type = ipywidgets.Dropdown(
    options=['sine', 'square', 'sawtooth', 'triangle'],
    value='sine',
    description='Type',
)

toggle_play_button = ipywidgets.ToggleButton(
    value=False,
    description="Start/Stop"
)

ipywidgets.jslink((freq_slider, 'value'), (osc.frequency, 'value'))
ipywidgets.jslink((detune_slider, 'value'), (osc.detune, 'value'))
ipywidgets.jslink((vol_slider, 'value'), (osc.volume, 'value'))
ipywidgets.link((osc_type, 'value'), (osc, 'type'))

def start_stop_osc(change):
    if change['new']:
        osc.start()
    else:
        osc.stop()

toggle_play_button.observe(start_stop_osc, names='value')

In [ ]:
display(toggle_play_button, osc_type, freq_slider, detune_slider, vol_slider)

### Get or control oscillator properties from within Python

In [ ]:
osc.frequency

In [ ]:
osc.type = "sawtooth"

In [ ]:
import time

osc.frequency.value = 100

for t in ["sine", "triangle", "square", "sawtooth"]:
    osc.type = t
    
    for f in range(100):
        time.sleep(0.01)
        osc.frequency.value += 1

In [ ]:
osc.stop()

### Schedule events along the timeline

In [ ]:
osc2 = ipytone.Oscillator()

In [ ]:
osc2.to_destination()

Let's schedule a callback invoked every eighth note after the first measure.

In [ ]:
def callback(time):
    osc2.start(time).stop(time + 0.1)

In [ ]:
event_id = ipytone.transport.schedule_repeat(callback, "8n", "1m")

The Transport has to be started for the callback to be invoked.

In [ ]:
ipytone.transport.start()

The event can be cancelled using `clear(event_id)`.

In [ ]:
ipytone.transport.clear(event_id)

In [ ]:
ipytone.transport.stop()

Add a callback event to a specific position along the Transport which will be invoked each time the Transport reaches that position.

In [ ]:
event_id = ipytone.transport.schedule(callback, "1m")

Again, the Transport has to be started.

In [ ]:
ipytone.transport.start()

When the Transport is stopped and restarted, the event is played again.

In [ ]:
ipytone.transport.stop()

In [ ]:
ipytone.transport.start()

Bip!

In [ ]:
ipytone.transport.stop()

In [ ]:
ipytone.transport.clear(event_id)

If this is not the intended behavior, you can use `schedule_once`.

In [ ]:
ipytone.transport.schedule_once(callback, "1m")

In [ ]:
ipytone.transport.start()

Bip!

In [ ]:
ipytone.transport.stop()

In this case, the event has been removed and it won't be played again.

In [ ]:
ipytone.transport.start()